In [2]:
import zipfile

# specify the path to the zip file
zip_path = 'logs_202212.zip'

# open the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
# list all files in the zip file
    zip_file_contents = zip_ref.namelist()

# extract all files in the zip file to a directory
    extract_directory = '/Users/kothaapuroop/Desktop/XN Project/extracted_files/' # replace with the name and path of the directory you want to extract to
    zip_ref.extractall(path=extract_directory)

In [34]:
# Automated

import os
import glob
import json
import re
import pandas as pd

base_folder = "/Users/kothaapuroop/Desktop/XN Project/extracted_files/logs_202212/"

# Get a list of all subfolders within the base folder
subfolders = [f.path for f in os.scandir(base_folder) if f.is_dir()]

for folder in subfolders:
    log_data = []

    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(".log"):
                file_path = os.path.join(root, file)
                with open(file_path, "r") as f:
                    for line in f:
                        match = re.match(r'(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}),(\w+),(\w+),({.+})', line)
                        if match:
                            timestamp, code, event_id, payload = match.groups()
                            try:
                                payload_json = json.loads(payload)
                                nested_json = json.loads(payload_json["b64Payload"])

                                log_data.append([
                                    timestamp, code, event_id,
                                    payload_json["MessageCode"],
                                    payload_json["EventId"],
                                    payload_json["EventTime"],
                                    payload_json["IngestTime"],
                                    nested_json["requestId"],
                                    nested_json["sourceIp"],
                                    nested_json["httpMethod"],
                                    nested_json["httpUrl"],
                                    nested_json["httpAuth"],
                                    nested_json["httpAuthHash"],
                                    nested_json["resource"],
                                    nested_json["resourceClass"],
                                    nested_json["resourceMethod"],
                                    nested_json["organization"],
                                    nested_json["app"],
                                    nested_json["user"],
                                    nested_json["entity"],
                                    nested_json["timestamp_req"],
                                    nested_json["timestamp_resp"]
                                ])
                            except (ValueError, KeyError):
                                print(f"Failed to parse payload: {payload}")
                        else:
                            print("No match found")

    df = pd.DataFrame(log_data, columns=[
        "Time Stamp", "Code", "Event ID", "Message Code", "Event ID", "Event Time", "Ingest Time", "Request ID",
        "Source IP", "HTTP Method", "HTTP Url", "HTTP Auth", "HTTP Auth Hash", "Resource", "Resource Class",
        "Resource Method", "Organization", "App", "User", "Entity", "Timestamp Request", "Timestamp Response"
    ])

    # Extract folder name from the path and use it to create the CSV file
    folder_name = os.path.basename(folder)
    output_path = os.path.join(base_folder, f"{folder_name}.csv")
    df.to_csv(output_path, index=False)

    print(f"Logs from folder {folder_name} written to {output_path}")


Logs from folder 20221220 written to /Users/kothaapuroop/Desktop/XN Project/extracted_files/logs_202212/20221220.csv
Logs from folder 20221218 written to /Users/kothaapuroop/Desktop/XN Project/extracted_files/logs_202212/20221218.csv
Logs from folder 20221227 written to /Users/kothaapuroop/Desktop/XN Project/extracted_files/logs_202212/20221227.csv
Logs from folder 20221211 written to /Users/kothaapuroop/Desktop/XN Project/extracted_files/logs_202212/20221211.csv
Logs from folder 20221229 written to /Users/kothaapuroop/Desktop/XN Project/extracted_files/logs_202212/20221229.csv
Logs from folder 20221216 written to /Users/kothaapuroop/Desktop/XN Project/extracted_files/logs_202212/20221216.csv
Logs from folder 20221228 written to /Users/kothaapuroop/Desktop/XN Project/extracted_files/logs_202212/20221228.csv
Logs from folder 20221217 written to /Users/kothaapuroop/Desktop/XN Project/extracted_files/logs_202212/20221217.csv
Logs from folder 20221210 written to /Users/kothaapuroop/Desktop